<table style="width:100%">
  <tr>
    <th align="right"><img src="img/dilogo.png" style="float:left;height:90px"></th>
    <th align="center">
    <h1 align="center">Tarea 0</h1>
    <h2 align="center">Profesor: Ricardo Ñanculef</h2>
    <h3 align="center">Ayudante: Fransisco Mena</h3>
    <h3 align="center">Camilo Sanchez Bravo</h3>
    <h3 align="center">201673586-0</h3>
    <h3 align="center">Gabriel Vergara Donoso</h3>
    <h3 align="center">201673605-0</h3>
    <h3 align="center">Septiembre 2019</h3>
    </th> 
    <th align="left"><img src="img/usmlogo.png" style="float:right;height:85px"></th>
  </tr>

<a id="primero"></a>
## 1. Perceptrón a mano

a) Escriba una función que calcule el valor de salida (output) del modelo  𝑓(𝑥)  para un patrón de entrada  𝑥  a través de los pesos  𝑤  del modelo. Decida si incluir los bias dentro de  𝑤  o manejarlos de manera separada.

 Implementación de la función f(x) en Python.

    Los bias se incluirán dentro de W, vector que contiene los pesos w. 
    Inputs:   
        X: vector que contiene los x.       
        W: vector que contiene los pesos w, donde w(i) corresponde al peso para el atributo x(i)    
    Output: Clase (1 o 0) que se calcula para X.

In [43]:
import numpy as np

def f(X,W):
    y = np.dot(X,W) #producto punto entre X y W
    if (y >= 0):
        return 1
    else:
        return 0

b) Escriba una función que implemente el clásico algoritmo del Perceptrón para un problema binario que permita entrenarlo en un conjunto de datos de tamaño  𝑁 , leídos de manera online (uno a uno). Recordar la decisión anterior sobre los bias.

In [44]:
def perceptron(Xs,Y):
    N = len(Xs[0]) ##cantidad de atributos
    pi = np.zeros(N) ##vector pi
    W = np.zeros(N) #vector W a retornar
    run_pi = run_w = i =0 #casos consecutivos correctos por pi o w, e i como contador
    for X in Xs: #aprende por caso
        y = f(X,pi) #clasificacion de X con los pesos pi
        if(y == Y[i]): 
            run_pi += 1
            if(run_pi > run_w): #
                W = np.copy(pi)
                run_w = run_pi
        else:
            pi = pi + X*Y[i] ##Se debe hacer bien el ajuste
            run_pi = 0
        i += 1
    return W
    
    

c) Demuestre que lo implementado funciona en un problema real de clasificación. Para esto utilice el dataset Breast cancer wisconsin, disponible a través de la librería sklearn, el cual corresponde a la detección de cancer mamario a través de características relevantes (numéricas continuas) de un examen realizado, como por ejemplo la textura, simetría y tamaño de una masa mamaria. Estas características deben combinarse linealmente para la detección del cancer.

In [45]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
X_train,y_train = load_breast_cancer(return_X_y=True)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
N,m=X_train.shape
X_train = np.c_[X_train, np.ones(N)]
i = 0
cantidad = len(y_train)
while(i<cantidad): #cambiar los 0 a -1 (necesarios en el perceptron)
    if (y_train[i]==0):
        y_train[i] = -1
    i +=1

pesos = perceptron(X_train, y_train)
correctas = 0
i = 0
#while(i<cantidad):
#    if(y_train[i]==f(X_train[i],pesos)):
#        correctas += 1
#    i +=1
#print(cantidad)
#print(correctas)

d) Escriba una función que implemente el **Forgetrón**  con una memoria de tamaño $K$ y la función de kernel como el producto interno (*inner-product*), esto es $<a,b> = \sum_i a_i \cdot b_i$.

In [48]:
def Forgetron(Xs,Y,K):
    N = len(Xs[0])
    f1 = np.zeros(N) #vector de pesos inicializados en 0s
    I1 = [] #memoria del algoritmo
    factores = np.zeros(K)
    Ys = [] #los 'y' de los datos que fallaron
    fallas = 0 
    pos = 0 #guarda la posicion actual en la memoria de los datos guardados
    i = 0 
    accu = []
    datos = []
    cantidad = len(Y)
    for X in Xs:
        if ((f(X,f1)*Y[i])<0): #si falla la prediccion
            #fallas += 1
            if (pos>=K): #por la memoria limitada
                pos = 0
            #I1[pos] = X
            if(fallas >= K): 
                Ys[pos] = Y[i]
                I1[pos] = X
                factores = factores*0.1
            else:
                Ys.append(Y[i])
                I1.append(X)
            factores[pos] = 1 #nuevo error
            pos += 1
            fallas +=1
        ft = np.zeros(N)
        for j in list(range(len(Ys))):
            ft = ft + factores[j]*Ys[j]*I1[j]
        f1 = np.copy(ft)
        i += 1
        datos.append(i)
        j = 0
        correctas = 0
        while(j<cantidad):
            if(Y[j]==f(Xs[j],f1)):
                correctas += 1.0
            j += 1
        accu.append(correctas/cantidad)
    plt.grid()
    plt.xlabel("Numero de datos de entrenamiento")
    plt.ylabel("Precision")
    plt.plot(datos,accu)
    return f1

e) Vuelva a realizar el item c) para el **Forgetrón** con un $K=10$ y compare los resultados.

In [47]:
pesos = Forgetron(X_train, y_train, 10)

NameError: name 'plt' is not defined

### ¿Qué sucede al variar la función objetivo del problema? 

f) Escriba una función que compute la función sigmoidal para una entrada  𝜉  cualquiera. Tenga cuidado con los límites de números que puede trabajar python (por ejemplo  exp800→+∞ ). Se aconseja acotar/truncar los valores que entran a la función para que la operación se pueda realizar. Además escriba una función que calcule la salida del nuevo modelo  𝑔(𝑥;𝑤,𝑏)  con esta función sigmoidal.

In [35]:
import math

def sigmoid(x):
    if x<-709:
        return 0
    return 1 / (1 + math.exp(-x))

def f2(X,W):
    y = np.dot(X,W) #producto punto entre X y W
    if (sigmoid(y) >= 0):
        return 1
    else:
        return 0

1.216780750623423e-308

g) Escriba una función que calcule la función de pérdida descrita anteriormente para un dato  𝑥(𝑖) , utilizando  𝑔(𝑥(𝑖);𝑤,𝑏) . Tenga cuidado con los límites del logaritmo (recordad que  log0→+∞ ).

In [ ]:
def bce(y,X,W):#y=f2(x,W)
    return -y*math.log(sigmoid(np.dot(X,W)))-(1-y)*math.log(1-sigmoid(np.dot(X,W)))

h) Escriba una función que calcule el gradiente (derivada) de la función de pérdida anterior con respecto a los pesos del modelo  𝑤 . Se recomienda derivarla analíticamente y luego escribirla. Recuerde el uso de la regla de la cadena.

In [9]:
def gradient(y,X,W):#sigmoid'=sigmoid*(1-sigmoid)
    aux=sigmoid(np.dot(X,W))
    return ( -y/aux + (1-y)/(1-aux) )*(aux)*(1-aux)*X

i) Realice una modificación al algoritmo implementado en b) (Perceptrón) para que se adapte a la función objetivo binary cross entropy implementada, para ésto haga uso del algoritmo de optimización SGD (Stochastic Gradient Descend) con tasa de aprendizaje  𝜂∈[0,1] .

In [ ]:
#w=w-etha*

def perceptron(Xs,Y):
    N = len(Xs[0]) ##cantidad de atributos
    pi = np.zeros(N) ##vector pi
    W = np.zeros(N) #vector W a retornar
    run_pi = run_w = i =0 #casos consecutivos correctos por pi o w, e i como contador
    for X in Xs: #aprende por caso
        y = f2(X,pi) #clasificacion de X con los pesos pi
        if(y == Y[i]): 
            run_pi += 1
            if(run_pi > run_w): #
                W = np.copy(pi)
                run_w = run_pi
        else:
            pi = pi + X*Y[i] ##Se debe hacer bien el ajuste
            run_pi = 0
        i += 1
    return W
    
    

j) Vuelva a realizar el item c) con esta modificación, además grafique la función de pérdida en el transcurso del entrenamiento. Compare los resultados.